In [2]:
import os
import cPickle as pickle
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [7]:
import pandas as pd
from app import db

In [20]:
CHURN_STATS_QUERY = """
SELECT DISTINCT 
    count(likely.distinct_id) AS churn_likely,
    count(possible.distinct_id) AS churn_possible,
    count(unlikely.distinct_id) AS churn_unlikely
FROM churns
LEFT JOIN (
    SELECT distinct_id
    FROM churns 
    WHERE churn_proba > 0.85
) AS likely
ON churns.distinct_id = likely.distinct_id

LEFT JOIN (
    SELECT distinct_id
    FROM churns 
    WHERE churn_proba < 0.85 AND churn_proba > 0.5
) AS possible
ON churns.distinct_id = possible.distinct_id

LEFT JOIN (
    SELECT distinct_id
    FROM churns
    WHERE churn_proba < 0.5
) AS unlikely
ON unlikely.distinct_id = churns.distinct_id
"""

CONVERSION_STATS_QUERY = """
SELECT DISTINCT 
    count(likely.distinct_id) AS likely_conversion,
    count(possible.distinct_id) AS possible_conversion,
    count(unlikely.distinct_id) AS unlikely_conversion
FROM conversions
LEFT JOIN (
    SELECT distinct_id
    FROM conversions
    WHERE conversion_proba > 0.85
) AS likely
ON conversions.distinct_id = likely.distinct_id

LEFT JOIN (
    SELECT distinct_id
    FROM conversions 
    WHERE conversion_proba < 0.85 AND conversion_proba > 0.5
) AS possible
ON conversions.distinct_id = possible.distinct_id

LEFT JOIN (
    SELECT distinct_id
    FROM churns
    WHERE churn_proba < 0.5
) AS unlikely
ON conversions.distinct_id = unlikely.distinct_id
"""

conversion_stats = pd.read_sql_query(CONVERSION_STATS_QUERY, db.engine)
churn_stats = pd.read_sql_query(CHURN_STATS_QUERY, db.engine)

stats_df = pd.concat([conversion_stats, churn_stats], axis=1)

dict(zip(stats_df.keys(), stats_df.values[0]))

{u'churn_likely': 18,
 u'churn_possible': 72,
 u'churn_unlikely': 1905,
 u'likely_conversion': 1198,
 u'possible_conversion': 2279,
 u'unlikely_conversion': 7}